In [1]:
import os
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

In [2]:
exp_time = [0, 2, 3, 4, 6, 8]*24
exp_outgrowth = [0, 60, 160, 450, 1280, 2260]
exp_outgrowth_err = [0, 10, 20, 40, 60, 70]

In [3]:
root = '/home/ecuracosta/Desktop/MS/Simulations/Models/Shrink-Shrink/'
os.chdir(root)

n0 = 205    # Number of cells after amputation
rl = -1045  # Recruitment limit
tv_array = np.arange(0,3*24+1,1)

dt_u = 1
penalty = 1

heatmap = pd.DataFrame(index=[n0],columns=tv_array)

for tv in tv_array:
    df_all = []
    parameters_set = "n0="+str(n0)+"_rl="+str(rl)+"_tv="+str(tv)
    files = os.listdir(parameters_set+'/Outgrowth')
    for seed in files:
        opened_file = open(parameters_set+'/Outgrowth'+"/"+seed)
        df0 = pd.read_csv(opened_file, delimiter=',')
        outgrowth = df0['outgrowth']*13.2
        df_all.append(outgrowth)
    df_all = pd.concat(df_all, ignore_index=True, axis=1)
    df_mean = np.asarray(df_all.mean(axis=1))
    df_std = np.asarray(df_all.std(axis=1))
    ssqi = []
    for outgrowth,error,time in zip(exp_outgrowth,exp_outgrowth_err,exp_time):        
        if df_mean[time*24]-dt_u*df_std[time*24] < outgrowth < df_mean[time*24]+dt_u*df_std[time*24]:
            ssqi.append(abs(df_mean[time*24]-outgrowth))   
        elif df_mean[time*24]-dt_u*df_std[time*24] < outgrowth-error < df_mean[time*24]+dt_u*df_std[time*24]:
            ssqi.append(abs(df_mean[time*24]-outgrowth))  
        elif df_mean[time*24]-dt_u*df_std[time*24] < outgrowth+error < df_mean[time*24]+dt_u*df_std[time*24]:
            ssqi.append(abs(df_mean[time*24]-outgrowth))
        else:
            ssqi.append(abs(df_mean[time*24]-outgrowth)*penalty)
    ssq = sum(ssqi)
    heatmap[tv] = ssq

In [4]:
heatmap

,0,1,2,3,4,5,6,7,8,9,...,63,64,65,66,67,68,69,70,71,72
205,1070.858,1036.934,1056.404,967.568,970.868,918.596,924.8,834.116,829.496,824.48,...,491.516,497.324,478.184,508.874,510.656,527.486,539.366,552.236,582.068,575.798


In [5]:
print(heatmap.min(axis=0).idxmin())

44


In [6]:
heatmap.min(axis=0)[44]

306.3040000000001